In [172]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import os 
import ast
import json

https://stackoverflow.com/questions/62335537/scraping-text-from-kickstarter-projects-return-nothing

In [173]:
base = "/Users/ivankoh/Downloads/kickstarter data/"
filepaths = [base + f for f in os.listdir(base) if f.endswith('.csv')]
df = pd.concat(map(pd.read_csv, filepaths))
df_1 = df.drop_duplicates(subset=['id'])
df_1['main_url'] = df_1['urls'].apply(lambda x: eval(x)['web']['project'])
print(len(df))
print(len(df_1))

392244
203510


/var/folders/dq/33rpc0cx4f54t0vx2cdcgczr0000gn/T/ipykernel_74056/4110031066.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['main_url'] = df_1['urls'].apply(lambda x: eval(x)['web']['project'])


In [201]:
def get_slug(url):
    return re.search('/projects/(.*)\?', url).group(1)

# get the slug from the urls. the slug is the input to the GraphQL API call 
slugs = list(map(get_slug, df_1['main_url'].tolist()))
# while iterating through all projects, add the story and risk to these lists. At the end len of both list 
# should be the same as the len of the dataframe. if all good then add as columns to the dataframe df. 
story_list = []
risk_list = []

# instantiate a new Session 
s = requests.Session()
headers = {
            'content-type': 'application/json',
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0',
           'cookie': '_ksr_session=fc2U7qXXaRN91foNiE53NyU3s181NZO0Ll57xPkYxZ5iyUNgus35a0HwsPBTfViBY%2ByAKbtpRirAVLxOGKzG%2BYMOmsLRBPujZep%2Fca%2B1%2FXzW3xX56VXkh5w6ItYhIctEFifQQhw3rTmvoljyHw%3D%3D--4pK6xBEgChjqgmte--LH4Q1qSnhU%2FYX9JgTzGuSQ%3D%3D;'
          }
# GET request to kickstarter to get some token needed for later 
# r = s.get("https://www.kickstarter.com", headers=headers)
# BeautifulSoup parses the html returned as a response from the GET request 
# soup = BeautifulSoup(r.text, 'html.parser')
# xcsrf = soup.find("meta", {"name": "csrf-token"})["content"]

query = """
query Campaign($slug: String!) {
  project(slug: $slug) {
    risks
    story(assetWidth: 680)
  }
}"""

for slug in slugs:
    print(f"--------{slug}------")
    # POST request to GraphQL API situated in kickstarter domain 
    # response from POST request should contain the story and risks
    r = s.post("https://www.kickstarter.com/graph",
        headers= {
             'content-type': 'application/json',
             'x-csrf-token': 'n9jjPXhH7XRF1Ee85FbBlXj-HI5fkNflWaVNY4JJhvZqu_gcjqls_UQg_f9YmNyVzogzvOEnuUiBCxw4Pjr6jQ',
             'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0',
             'cookie': '_ksr_session=Z9sXOXoSL%2Foru75CmU3CcT4D5kuUd97J7QkENXgLec8FNahKWe5RQlJQHsLrnSzgaqKmeMw99LCmLc5wRyKwhB%2BzNg0YKOcMlAl%2FTk41xFRIIYvcgZ%2FKpQaz0GTkAGL5mHgvsMiEpplG3kHi5UJ%2Fg2f7UjyQJQ7XpilzX8ICW7If9ts%2BltTO8PHbIdcnOK%2FNSPlTYtGR2Y%2BzE%2F89ke1hzQM5SWhEd2t%2BTGhGPZYtdU8oBix6b6ot2wiJHqDJ5ZQwrq3%2Bne6X3YqXDIosTqMYMjDfKOc%3D--M6qDkunMrGDU2OvV--s7768lOwz%2BWKhBCjwtv78g%3D%3D'
        },
        json = {
            "operationName":"Campaign",
            "variables":{
                "slug": slug
            },
            "query": query
        })
    try:
        result = r.json()
        print("-------STORY--------")
        story_html = result["data"]["project"]["story"]
        soup = BeautifulSoup(story_html, 'html.parser')
        curr_story = []
        for i in soup.find_all('p'):
            print(i.get_text())
            curr_story.append(i.get_text())
        story_list.append(" ".join(curr_story) if curr_story else "")

        print("-------RISKS--------")
        print(result["data"]["project"]["risks"])
        risk_list.append(result["data"]["project"]["risks"] if result["data"]["project"]["risks"] else "")
    except:
        print(r)
        story_list.append("")
        risk_list.append("")
        continue

--------ademibar/ademi-plant-based-protein-bar------
<Response [403]>
--------unicoxsuperfoods/unicox-superfoods-performance-breakfast-blend------
<Response [403]>
--------975599626/psycho-zombies-film-funding------
<Response [403]>
--------110466015/oil-paintings-of-new-yorks-burlesque------
<Response [403]>
--------53769410/awareness-through-photography------
<Response [403]>
--------747924081/the-battle-of-success-a-business-going-nowhere------
<Response [403]>
--------1422772588/the-ghost-with-in------
<Response [403]>
--------1722483947/the-cult-of-ammon------
<Response [403]>
--------29154331/decisions-is-a-short-horror-film-from-the-creators------
<Response [403]>
--------leebobawitz/i-ship-it-too-another-fanfic-enamel-pin-collection------
<Response [403]>
--------1869715485/still-alive-1------
<Response [403]>
--------1457926799/dead-end-1------
<Response [403]>
--------andresgomz/la-belleza-de-la-naturaleza-xalapena------
<Response [403]>
--------865600006/cosmos-sunshine-new-

KeyboardInterrupt: 

In [165]:
s = requests.Session()

In [203]:
import requests
import json

url = "https://www.kickstarter.com/graph"

payload = json.dumps([
  {
    "operationName": "Campaign",
    "variables": {
      "slug": "leightonconnor/akashic-titan-blue-bolt"
    },
    "query": "query Campaign($slug: String!) {\n  project(slug: $slug) {\n    id\n    isSharingProjectBudget\n    risks\n    story(assetWidth: 680)\n    currency\n    spreadsheet {\n      displayMode\n      public\n      url\n      data {\n        name\n        value\n        phase\n        rowNum\n        __typename\n      }\n      dataLastUpdatedAt\n      __typename\n    }\n    environmentalCommitments {\n      id\n      commitmentCategory\n      description\n      __typename\n    }\n    __typename\n  }\n}\n"
  }
])
headers = {
  'content-type': 'application/json',
  'x-csrf-token': 'n9jjPXhH7XRF1Ee85FbBlXj-HI5fkNflWaVNY4JJhvZqu_gcjqls_UQg_f9YmNyVzogzvOEnuUiBCxw4Pjr6jQ',
  'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0',
  'cookie': '_ksr_session=j%2FWnLAz8hLN7C7FuR3oVOuOJz%2BKiUl2x9ghHoHeG8IeZbXPM2Ly%2Fvqlt64YjLuBtQw8AJUh1gj%2BAJfsXO4ie20Roh59fg9I9331K1NtcDc1ENXbxlLLDvrm%2FE8gsyTqBCcKIDhiq5L6FKheZPkn%2B%2BMWf5QoeXLAPQoJVHedLiSQwgIo7XaQqe7baaJwdsm%2FdApdUqj1TtbnMvplGF28cmWvS5FU90BOpwW2%2Bep6UXNzMbKllfI%2FFpbPgzAima7kd1jMIsQ0QhQZc4TupVKOxHlJlGww%3D--LZdmdnIjdcwChj87--k1ngQVQlcDRhIKa6KjeuBA%3D%3D'
}

response = requests.request("POST", url, headers=headers, data=payload)
print(response.status_code)
print(response.json()[0]['data']['project']['risks'])

403


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [198]:
import requests
import json

url = "https://www.kickstarter.com/graph"

payload = json.dumps([
  {
    "operationName": "Campaign",
    "variables": {
      "slug": "leightonconnor/akashic-titan-blue-bolt"
    },
    "query": "query Campaign($slug: String!) {\n  project(slug: $slug) {\n    id\n    isSharingProjectBudget\n    risks\n    story(assetWidth: 680)\n    currency\n    spreadsheet {\n      displayMode\n      public\n      url\n      data {\n        name\n        value\n        phase\n        rowNum\n        __typename\n      }\n      dataLastUpdatedAt\n      __typename\n    }\n    environmentalCommitments {\n      id\n      commitmentCategory\n      description\n      __typename\n    }\n    __typename\n  }\n}\n"
  }
])
headers = {
  'content-type': 'application/json',
  'x-csrf-token': 'AZsT67Z9s-LHZt6ZJXLSQWJlNdd7biKz2XDfFMkcYMZrNufH1OWoFhNBlXIvxCrxKRzV6l8bG_Z6QlcRoYMe_g',
  'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36',
  'cookie': '_ksr_session=fc2U7qXXaRN91foNiE53NyU3s181NZO0Ll57xPkYxZ5iyUNgus35a0HwsPBTfViBY%2ByAKbtpRirAVLxOGKzG%2BYMOmsLRBPujZep%2Fca%2B1%2FXzW3xX56VXkh5w6ItYhIctEFifQQhw3rTmvoljyHw%3D%3D--4pK6xBEgChjqgmte--LH4Q1qSnhU%2FYX9JgTzGuSQ%3D%3D;'
}

response = requests.request("POST", url, headers=headers, data=payload)
print(response.status_code)
print(response.json()[0]['data']['project']['risks'])

200
This is my fourth solo Kickstarter campaign, and I delivered all three of my previous games in a timely manner. 

I have a complete draft of the text, with the exception of Appendix 1. I have a team of editors lined up who will give me feedback so that I can revise, expand, and polish it as needed. The most time-consuming tasks will be completing the artwork and the layout. I've done this before, with the previous Akashic Titan zine and with my game Project 8Ball. Depending on the stretch goals I will be depending on someone else for illustrations and additional text, but I have worked with Josh before and I am confident he will hit his deadlines. 

I believe I will be able to finish the book by June, but just to be cautious I have set the estimated delivery month as July. 

Barring any unforeseen catastrophes, I should have no problem printing and shipping the zine by July.
